# Day 19 - Back to that CPU

* [Day 19](https://adventofcode.com/2018/day/19)

We return to the CPU from [day 16](./Day%2016.ipynb); I remarked then (in my part 2 notes) that executing instructions was *certainly going to be straightforward, as there are no jumps (so no loops) in this instruction set.* Now we have jumps!

In [1]:
import operator
import re
from dataclasses import dataclass
from functools import partial
from typing import Callable, Iterable, Iterator, List, Mapping, Set, Sequence, Tuple, Type, Union

Instruction = Tuple[str, int, int, int]
OpcodeFunction = Union[Callable[[int, int], int], Callable[[int], int]]

@dataclass
class Opcode:
    # An opcode function takes one or two integers as input
    f: OpcodeFunction
    operands: str # 1 or 2 characters, i and r
    opcode: str = ''

    def __set_name__(self, owner: Type['CPU'], name: str) -> None:
        self.opcode = name[3:]
        owner.opcodes[self.opcode] = self

    def __repr__(self) -> str:
        return f'<opcode {self.opcode} {self.operands!r}>'
    
    def __get__(self, instance: 'CPU', owner: Type['CPU']) -> Callable[[int, int, int], None]:
        return partial(self.__call__, instance)

    def __call__(self, cpu: 'CPU', *ops: int) -> None:
        # operand C receives the output
        *ops, op_c = ops
        # map the oter operands to values
        value = {'r': cpu.registers.__getitem__, 'i': operator.pos}
        ops = (value[t](op) for t, op in zip(self.operands, ops))
        # execute the operand, store the integer result in the register
        # designated by C. Int is used to convert bool results to 0 or 1
        cpu.registers[op_c] = int(self.f(*ops))

def opcode(operands: str) -> Callable[[OpcodeFunction], Opcode]:
    """Operator operates on 1 or 2 operands
    
    Specify the operands as 'r' or 'i' for registers and immediate values.
    """
    def decorator(f: OpcodeFunction) -> Opcode:
        return Opcode(f, operands)
    return decorator

class CPU:
    registers: List[int]  # limited to 6
    opcodes: Mapping[str, Opcode] = {}

    def __init__(self) -> None:
        self.registers = [0, 0, 0, 0, 0, 0]
        self.bound: Mapping[str, Callable[[int, int, int], None]] = {
            name: opcode.__get__(self, None) for name, opcode in CPU.opcodes.items()
        }
    
    # Addition
    # addr (add register) stores into register C the result of adding register
    # A and register B.
    op_addr = opcode('rr')(operator.add)
    # addi (add immediate) stores into register C the result of adding
    # register A and value B.
    op_addi = opcode('ri')(operator.add)

    # Multiplication:
    # mulr (multiply register) stores into register C the result of
    # multiplying register A and register B.
    op_mulr = opcode('rr')(operator.mul)
    # muli (multiply immediate) stores into register C the result of
    # multiplying register A and value B.
    op_muli = opcode('ri')(operator.mul)

    # Bitwise AND:
    # banr (bitwise AND register) stores into register C the result of the
    # bitwise AND of register A and register B.
    op_banr = opcode('rr')(operator.and_)
    # bani (bitwise AND immediate) stores into register C the result of the
    # bitwise AND of register A and value B.
    op_bani = opcode('ri')(operator.and_)

    # Bitwise OR:
    # borr (bitwise OR register) stores into register C the result of the
    # bitwise OR of register A and register B.
    op_borr = opcode('rr')(operator.or_)
    # bori (bitwise OR immediate) stores into register C the result of the
    # bitwise OR of register A and value B.
    op_bori = opcode('ri')(operator.or_)

    # Assignment:
    # setr (set register) copies the contents of register A into register C.
    # (Input B is ignored.)
    op_setr = opcode('r')(operator.pos)  # pos is the identity function for int
    # seti (set immediate) stores value A into register C. (Input B is
    # ignored.)
    op_seti = opcode('i')(operator.pos)  # pos is the identity function for int

    # Greater-than testing:
    # gtir (greater-than immediate/register) sets register C to 1 if value A
    # is greater than register B. Otherwise, register C is set to 0.
    op_gtir = opcode('ir')(operator.gt)
    # gtri (greater-than register/immediate) sets register C to 1 if register
    # A is greater than value B. Otherwise, register C is set to 0.
    op_gtri = opcode('ri')(operator.gt)
    # gtrr (greater-than register/register) sets register C to 1 if register A
    # is greater than register B. Otherwise, register C is set to 0.
    op_gtrr = opcode('rr')(operator.gt)

    # Equality testing:
    # eqir (equal immediate/register) sets register C to 1 if value A is equal
    # to register B. Otherwise, register C is set to 0.
    op_eqir = opcode('ir')(operator.eq)
    # eqri (equal register/immediate) sets register C to 1 if register A is
    # equal to value B. Otherwise, register C is set to 0.
    op_eqri = opcode('ri')(operator.eq)
    # eqrr (equal register/register) sets register C to 1 if register A is
    # equal to register B. Otherwise, register C is set to 0.
    op_eqrr = opcode('rr')(operator.eq)

    def execute(self, ipregister: int, instructions: Sequence[Instruction]) -> int:
        opcodes = self.bound
        r = self.registers
        while 0 <= r[ipregister] < len(instructions):
            op, *operands = instructions[r[ipregister]]
            opcodes[op](*operands)
            r[ipregister] += 1
        return self.registers[0]

def parse_instructions(lines: Iterable[str]) -> Tuple[int, Sequence[Instruction]]:
    it = iter(lines)
    ipregister = int(next(it).rpartition(' ')[-1])
    instructions = []
    for line in it:
        opcode, *operands = line.split()
        instructions.append((opcode, *map(int, operands)))
    return ipregister, instructions

In [2]:
testprogram = parse_instructions('''\
#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5'''.splitlines())
assert CPU().execute(*testprogram) == 7

In [3]:
import aocd

data = aocd.get_data(day=19, year=2018)
ipregister, instructions = parse_instructions(data.splitlines())

In [4]:
print('Part 1:', CPU().execute(ipregister, instructions))

Part 1: 912


## Part 2, analysis

The puzzle is specifically crafted to take a very, very long time once you set register 0 to 1. We need to find a shortcut. The machine code given isn't very long, we need to analyse it a little to see what it does and were we can cut a corner.

Looking over my puzzle input, I annotated the instructions with my interpretation; my IP is register #4. If you are looking at this notebook on Github, you may want to [switch to the Jupyter notebook viewer](https://nbviewer.jupyter.org/github/mjpieters/adventofcode/blob/master/2018/Day%2019.ipynb) instead to see my colour annotations and have label links work:

<table>
  <thead>
    <tr>
      <th align="left" style="text-align: left">label</th>
      <th align="right">i</th>
      <th>opcode</th>
      <th align="right">op1</th>
      <th align="right">op2</th>
      <th align="right">reg</th>
      <th align="left" style="text-align: left">interpretation</th>
    </tr>
  </thead>
  <tbody>
    <tr id="label-SETUP">
      <td align="left"></td>
      <td align="right">0</td>
      <td>addi</td>
      <td align="right">4</td>
      <td align="right">16</td>
      <td align="right">4</td>
      <td align="left" style="text-align: left"><code>jmp 17 <a href="#label-SETUP" style="color: mediumblue; text-decoration: none; font-style: italic">(SETUP)</a></code></td>
    </tr>
    <tr><td colspan="7"></td></tr>
  </tbody>
  <tbody>
    <tr id="label-MAIN">
      <td align="left" style="text-align: left; color: teal"><strong>MAIN</strong></td>
      <td align="right">1</td>
      <td>seti</td>
      <td align="right">1</td>
      <td align="right"></td>
      <td align="right">2</td>
      <td align="left" style="text-align: left"><code>r2 = 1</code></td>
    </tr>
    <tr id="label-LOOP1" style="border-top: solid green">
      <td align="left" style="text-align: left; color: green; border-left: solid green"><strong>LOOP1</strong></td>
      <td align="right" style="border-top: solid green">2</td>
      <td>seti</td>
      <td align="right">1</td>
      <td align="right"></td>
      <td align="right">5</td>
      <td align="left" style="text-align: left"><code>r5 = 1</code></td>
    </tr>
    <tr id="label-LOOP2" style="border-left: solid green">
      <td align="left" style="text-align: left; color: purple"><strong>LOOP2</strong></td>
      <td align="right" style="border-left: solid purple; border-top: solid purple">3</td>
      <td style="border-top: solid purple">mulr</td>
      <td align="right" style="border-top: solid purple">2</td>
      <td align="right" style="border-top: solid purple">5</td>
      <td align="right" style="border-top: solid purple">3</td>
      <td align="left" rowspan="5" style="text-align: left; vertical-align: middle; border-top: solid purple">
        <pre><code>if (r2 * r5) == r1:
    r0 += r2</code></pre>
      </td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right" style="border-left: solid purple">4</td>
      <td>eqrr</td>
      <td align="right">3</td>
      <td align="right">1</td>
      <td align="right">3</td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right" style="border-left: solid purple">5</td>
      <td>addr</td>
      <td align="right">3</td>
      <td align="right">4</td>
      <td align="right">4</td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right" style="border-left: solid purple">6</td>
      <td>addi</td>
      <td align="right">4</td>
      <td align="right">1</td>
      <td align="right">4</td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right" style="border-left: solid purple">7</td>
      <td>addr</td>
      <td align="right">2</td>
      <td align="right">0</td>
      <td align="right">0</td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right" style="border-left: solid purple">8</td>
      <td>addi</td>
      <td align="right">5</td>
      <td align="right">1</td>
      <td align="right">5</td>
      <td align="left" style="text-align: left"><code>r5 += 1</code></td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right" style="border-left: solid purple">9</td>
      <td>gtrr</td>
      <td align="right">5</td>
      <td align="right">1</td>
      <td align="right">3</td>
      <td align="left" rowspan="3" style="vertical-align: middle; text-align: left; border-bottom: solid purple">
        <pre><code>if r5 &lt;= r1:
    jmp 3 (<a href="#label-LOOP2"style="color: purple; text-decoration: none; font-style: italic">LOOP2</a>)</code></pre>
      </td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right" style="border-left: solid purple">10</td>
      <td>addr</td>
      <td align="right">4</td>
      <td align="right">3</td>
      <td align="right">4</td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right" style="border-left: solid purple; border-bottom: solid purple">11</td>
      <td style="border-bottom: solid purple">seti</td>
      <td align="right" style="border-bottom: solid purple">2</td>
      <td align="right" style="border-bottom: solid purple"></td>
      <td align="right" style="border-bottom: solid purple">4</td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right">12</td>
      <td>addi</td>
      <td align="right">2</td>
      <td align="right">1</td>
      <td align="right">2</td>
      <td align="left" style="text-align: left"><code>r2 += 1</code></td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right">13</td>
      <td>gtrr</td>
      <td align="right">2</td>
      <td align="right">1</td>
      <td align="right">3</td>
      <td align="left" rowspan="3" style="vertical-align: middle; text-align: left">
        <pre><code>if r2 &lt;= r1:
    jmp 2 (<a href="#label-LOOP1"style="color: green; text-decoration: none; font-style: italic">LOOP1</a>)</code></pre></td>
    </tr>
    <tr style="border-left: solid green">
      <td align="left"></td>
      <td align="right">14</td>
      <td>addr</td>
      <td align="right">3</td>
      <td align="right">4</td>
      <td align="right">4</td>
    </tr>
    <tr style="border-left: solid green; border-bottom: solid green">
      <td align="left"></td>
      <td align="right">15</td>
      <td>seti</td>
      <td align="right">1</td>
      <td align="right"></td>
      <td align="right">4</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">16</td>
      <td>mulr</td>
      <td align="right">4</td>
      <td align="right">4</td>
      <td align="right">4</td>
      <td align="left" style="text-align: left"><strong>HALT</strong></td>
    </tr>
    <tr><td colspan="7"></td></tr>
  </tbody>
  <tbody>
    <tr id="label-SETUP">
      <td align="left" style="text-align: left; color: darkblue"><strong>SETUP</strong></td>
      <td align="right">17</td>
      <td>addi</td>
      <td align="right">1</td>
      <td align="right">2</td>
      <td align="right">1</td>
      <td align="left" rowspan="8" style="vertical-align: middle; text-align: left">
        <code>r1 = 911</code>
        <p>
          <em>simplified from<br>
            <pre><code>r1 = 2 ** 2 * 19 * 11 = 836
r3 = 3 * 22 + 9 = 75
r1 += r3</code></pre>
          </em>
        </p>
      </td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">18</td>
      <td>mulr</td>
      <td align="right">1</td>
      <td align="right">1</td>
      <td align="right">1</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">19</td>
      <td>mulr</td>
      <td align="right">4</td>
      <td align="right">1</td>
      <td align="right">1</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">20</td>
      <td>muli</td>
      <td align="right">1</td>
      <td align="right">11</td>
      <td align="right">1</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">21</td>
      <td>addi</td>
      <td align="right">3</td>
      <td align="right">3</td>
      <td align="right">3</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">22</td>
      <td>mulr</td>
      <td align="right">3</td>
      <td align="right">4</td>
      <td align="right">3</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">23</td>
      <td>addi</td>
      <td align="right">3</td>
      <td align="right">9</td>
      <td align="right">3</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">24</td>
      <td>addr</td>
      <td align="right">1</td>
      <td align="right">3</td>
      <td align="right">1</td>
    </tr>
    <tr><td colspan="7"></td></tr>
  </tbody>
  <tbody>
    <tr>
      <td align="left"></td>
      <td align="right">25</td>
      <td>addr</td>
      <td align="right">4</td>
      <td align="right">0</td>
      <td align="right">4</td>
      <td align="left" rowspan="2" style="vertical-align: middle; text-align: left">
        <pre><code>if r0 == 0:
    jmp 1 (<a href="#label-MAIN"style="color: teal; text-decoration: none; font-style: italic">MAIN</a>)</code></pre></td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">26</td>
      <td>seti</td>
      <td align="right">0</td>
      <td align="right"></td>
      <td align="right">4</td>
    </tr>
    <tr><td colspan="7"></td></tr>
  </tbody>
  <tbody>
    <tr id="label-PART2">
      <td align="left" style="text-align: left; color: cadetblue"><strong>PART2</strong></td>
      <td align="right">27</td>
      <td>setr</td>
      <td align="right">4</td>
      <td align="right"></td>
      <td align="right">3</td>
      <td align="left" rowspan="7" style="vertical-align: middle; text-align: left">
        <code>r1 = 10551311</code>
        <p>
          <em>simplified from
            <pre><code>r3 = (27 * 28 + 29) * 30 * 14 * 32 = 10550400
r1 += r3</code></pre>
          </em>
        </p>
      </td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">28</td>
      <td>mulr</td>
      <td align="right">3</td>
      <td align="right">4</td>
      <td align="right">3</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">29</td>
      <td>addr</td>
      <td align="right">4</td>
      <td align="right">3</td>
      <td align="right">3</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">30</td>
      <td>mulr</td>
      <td align="right">4</td>
      <td align="right">3</td>
      <td align="right">3</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">31</td>
      <td>muli</td>
      <td align="right">3</td>
      <td align="right">14</td>
      <td align="right">3</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">32</td>
      <td>mulr</td>
      <td align="right">3</td>
      <td align="right">4</td>
      <td align="right">3</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">33</td>
      <td>addr</td>
      <td align="right">1</td>
      <td align="right">3</td>
      <td align="right">1</td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">34</td>
      <td>seti</td>
      <td align="right">0</td>
      <td align="right"></td>
      <td align="right">0</td>
      <td align="left" style="text-align: left"><code>r0 = 0</code></td>
    </tr>
    <tr>
      <td align="left"></td>
      <td align="right">35</td>
      <td>seti</td>
      <td align="right">0</td>
      <td align="right"></td>
      <td align="right">4</td>
      <td align="left" style="text-align: left"><code>jmp 36 (<a href="#label-MAIN"style="color: teal; text-decoration: none; font-style: italic">MAIN</a>)</code></td>
    </tr>
  </tbody>
</table>

The above can be condensed into the following Python code:

```python
v = 911 if part1 else 10551311
result = 0
for i in range(1, v + 1):
    for j in range(1, v + 1):
        if i * j == v:
            result += i
```

So we are basically finding the sum of the divisors of the given number. For 911, a prime number, that's just 911 itself and 1, and so the answer is 912.

For part 2 the number becomes a lot, lot larger, so lets not bother with our CPU at all and just calculate the number using Python. Not with the above code, of course; we don't need a double loop here!

In [5]:
from itertools import chain

def sum_factors(n):
    return sum(set(chain.from_iterable((i, n // i) for i in range(1, int(n**0.5) + 1) if not n % i)))

assert sum_factors(911) == 912

In [6]:
print('Part 2:', sum_factors(10551311))

Part 2: 10576224
